[DIY Covid-19 Dashboard Kit](https://github.com/fsavundra/Projects) (C) Freddie Savundra, 2020 ([f.savundra@se20.qmul.ac.uk](mailto:f.savundra@se20.qmul.ac.uk) - [web](https://www.linkedin.com/in/freddie-savundra-729679131/?originalSubdomain=uk)). All rights reserved.

# Covid-19 Dashboard

The dashboard attached below references two statistics that have been examined in this dashboard. In the first instance, we examine the relationship between the number of hospital cases and the number of deaths over a 28 day period. In the second instance, we examine the number of hospital admissions and the relationship between the number of ventilators issued.

In [1]:
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [2]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

## Relationship between the number of cases and death rates


In [3]:
filters = [
    'areaType=nation',
    'areaName=England'
]


# values here are the names of the PHE metrics
structure = {
     "date": "date",
     "admissions": "newAdmissions",
     "ventilators": "covidOccupiedMVBeds"
}

In [4]:
api = Cov19API(filters=filters, structure=structure)

In [5]:
# NOTE: this call polls the server. It may fail in case of connectivity problems or if the data
# are not available for any reason. It will also fail if the metric in the structure are not compatible
# with the filters (eg they are not defined at the national or local level).
agedistribution=api.get_json()

In [6]:
filters = [
    'areaType=nation',
    'areaName=England'
    ]

In [7]:
structure = {
    "date": "date",
    "cases": "hospitalCases",
    "deaths": "newDeaths28DaysByDeathDate"    
}

In [8]:
api = Cov19API(filters=filters, structure=structure)

In [9]:
# NOTE: this call polls the server. It may fail in case of connectivity problems or if the data
# are not available for any reason. It will also fail if the metric in the structure are not compatible
# with the filters (eg they are not defined at the national or local level).
timeseries=api.get_json()

In [10]:
import json

In [11]:
with open("agedistribution.json", "wt") as OUTF:
    json.dump(agedistribution, OUTF)

In [12]:
with open("timeseries.json", "wt") as OUTF:
    json.dump(timeseries, OUTF)

In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import json

In [14]:
# an iPython  "magic" that enables the embedding of matplotlib output
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [15]:
with open("timeseries.json", "rt") as INFILE:
    data=json.load(INFILE)

In [16]:
datalist=data['data']


In [17]:
dates=[dictionary['date'] for dictionary in datalist ]
dates.sort()


In [18]:
def parse_date(datestring):
    """ Convert a date string into a pandas datetime object """
    return pd.to_datetime(datestring, format="%Y-%m-%d")

In [19]:
startdate=parse_date(dates[0])
enddate=parse_date(dates[-1])

In [20]:
index=pd.date_range(startdate, enddate, freq='D')
timeseriesdf=pd.DataFrame(index=index, columns=['cases', 'deaths',])


In [21]:
for entry in datalist: # each entry is a dictionary with date, cases, hospital and deaths
    date=parse_date(entry['date'])
    for column in ['cases', 'deaths']:
        # check that nothing is there yet - just in case some dates are duplicated,
        # maybe with data for different columns in each entry
        if pd.isna(timeseriesdf.loc[date, column]): 
            # replace None with 0 in our data 
            value= float(entry[column]) if entry[column]!=None else 0.0
            # this is the way you access a specific location in the dataframe - use .loc
            # and put index,column in a single set of [ ]
            timeseriesdf.loc[date, column]=value
            
# fill in any remaining "holes" due to missing dates
timeseriesdf.fillna(0.0, inplace=True)
            


In [22]:
with open("agedistribution.json", "rt") as INFILE:
    data=json.load(INFILE)

In [23]:
datalist=data['data']


In [24]:
dates=[dictionary['date'] for dictionary in datalist ]
dates.sort()


In [25]:
def parse_date(datestring):
    """ Convert a date string into a pandas datetime object """
    return pd.to_datetime(datestring, format="%Y-%m-%d")

In [26]:
startdate=parse_date(dates[0])
enddate=parse_date(dates[-1])

In [27]:
index=pd.date_range(startdate, enddate, freq='D')
agedistributiondf=pd.DataFrame(index=index, columns=['admissions', 'ventilators'])


In [28]:
for entry in datalist: # each entry is a dictionary with date, cases, hospital and deaths
    date=parse_date(entry['date'])
    for column in ['admissions', 'ventilators']:
        # check that nothing is there yet - just in case some dates are duplicated,
        # maybe with data for different columns in each entry
        if pd.isna(agedistributiondf.loc[date, column]): 
            # replace None with 0 in our data 
            value= float(entry[column]) if entry[column]!=None else 0.0
            # this is the way you access a specific location in the dataframe - use .loc
            # and put index,column in a single set of [ ]
            agedistributiondf.loc[date, column]=value
            
# fill in any remaining "holes" due to missing dates
agedistributiondf.fillna(0.0, inplace=True)
            


In [29]:
timeseriesdf.to_pickle("timeseriesdf.pkl")

In [30]:
agedistributiondf.to_pickle("agedistributiondf.pkl")

In [31]:
import ipywidgets as wdg
import pandas as pd
import matplotlib.pyplot as plt

In [32]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [33]:
timeseriesdf=pd.read_pickle("timeseriesdf.pkl")

In [34]:
series=wdg.SelectMultiple(
    options=['cases', 'deaths'],
    value=['cases', 'deaths'],
    rows=2,
    description='Stats:',
    disabled=False
)

scale=wdg.RadioButtons(
    options=['linear', 'log'],
#    value='pineapple', # Defaults to 'pineapple'
#    layout={'width': 'max-content'}, # If the items' names are long
    description='Scale:',
    disabled=False
)

# try replacing HBox with a VBox
controls=wdg.HBox([series, scale])

def timeseries_graph(gcols, gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        timeseriesdf[list(gcols)].plot(logy=logscale)
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

# keep calling timeseries_graph(gcols=value_of_series, gscale=value_of_scale); capture output in variable graph   
graph=wdg.interactive_output(timeseries_graph, {'gcols': series, 'gscale': scale})

display(controls, graph)

Output()

In [35]:
# our API access function. This will be called by the button when it is clicked
def access_api():
    # Ignore the parameter, put code for polling the API here
     filters = [
    'areaType=nation',
    'areaName=England'
    ]

     structure = {
     "date": "date",
     "cases": "hospitalCases",
     "deaths": "newDeaths28DaysByDeathDate"  
    }

     api = Cov19API(filters=filters, structure=structure)
     timeseries=api.get_json()
     return timeseries
     
        
apibutton=wdg.Button(
     description='Refresh',
     disabled=False,
     button_style='', # 'success', 'info', 'warning', 'danger' or ''
     tooltip='Click to download current Public Health England data',
     icon='download' # (FontAwesome names without the `fa-` prefix)
)

# register the callback function with the button
def api_button_callback (button):
     apidata=access_api()
     global df_1
     timeseriesdf=wrangle_data(apidata)
     apibutton.icon="check"

apibutton.on_click(api_button_callback)

display(apibutton)


Button(description='Refresh', icon='download', style=ButtonStyle(), tooltip='Click to download current Public …

TypeError: access_api() missing 1 required positional argument: 'button'

## Relationship between Hospital Admissions and Ventilators Issued

In [36]:
agedistributiondf=pd.read_pickle("agedistributiondf.pkl")

In [37]:
# our API access function. This will be called by the button when it is clicked
def access_api(button):
    # Ignore the parameter, put code for polling the API here
    print("Please wait while I reach out to the Government Statistics Page...")
    print("...The Dashboard has successfully refreshed.")
    
# see the doc for the parameters    
apibutton=wdg.Button(
    description='Refresh',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to download current Public Health England data',
    icon='download' # (FontAwesome names without the `fa-` prefix)
)

# register the callback function with the button
apibutton.on_click(access_api)

# this is an iPython function that generalises print for Jupyter Notebooks; we use it to 
# display the widgets
display(apibutton)


Button(description='Refresh', icon='download', style=ButtonStyle(), tooltip='Click to download current Public …

In [38]:
series=wdg.SelectMultiple(
    options=['admissions', 'ventilators'],
    value=['admissions', 'ventilators'],
    rows=2,
    description='Stats:',
    disabled=False
)

scale=wdg.RadioButtons(
    options=['linear', 'log'],
#    value='pineapple', # Defaults to 'pineapple'
#    layout={'width': 'max-content'}, # If the items' names are long
    description='Scale:',
    disabled=False
)

# try replacing HBox with a VBox
controls=wdg.HBox([series, scale])

def agedistribution_graph(gcols, gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        agedistributiondf[list(gcols)].plot(logy=logscale)
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

# keep calling timeseries_graph(gcols=value_of_series, gscale=value_of_scale); capture output in variable graph   
graph=wdg.interactive_output(agedistribution_graph, {'gcols': series, 'gscale': scale})

display(controls, graph)

Output()

[DIY Covid-19 Dashboard Kit](https://github.com/fsavundra/Projects) (C) Freddie Savundra, 2020 ([f.savundra@se20.qmul.ac.uk](mailto:f.savundra@se20.qmul.ac.uk) - [web](https://www.linkedin.com/in/freddie-savundra-729679131/?originalSubdomain=uk)). All rights reserved. Based on UK Government data published by Public Health England.